###Baseline Model

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
def ts_split(series, test_size=0.2):
    split = int(len(series) * (1 - test_size))
    return series[:split], series[split:]

In [ ]:
all_results = []

group_cols = ['Category', 'Region', 'Store ID', 'Product ID']

In [ ]:
#holt winter
for (cat, reg, store, prod), grp in df.groupby(group_cols):

    ts = grp.sort_values('Date').set_index('Date')['Demand']

    if len(ts) < 40:
        continue

    train, test = ts_split(ts)

        # --------------------------------------
    # Holt-Winters Exponential Smoothing
    # --------------------------------------
    try:
        hw_model = ExponentialSmoothing(
            train,
            trend='add',
            seasonal='add',
            seasonal_periods=12
        )

        hw_fit = hw_model.fit()
        hw_forecast = hw_fit.forecast(len(test))

        all_results.append({
            'Model': 'Holt-Winters',
            'Category': cat,
            'Region': reg,
            'Store ID': store,
            'Product ID': prod,
            'MAPE': mean_absolute_percentage_error(test, hw_forecast),
            'R2': r2_score(test, hw_forecast)
        })
    except Exception as e:
        pass

###Fine-tuned Model

In [ ]:
df['Demand_log'] = np.log1p(df['Demand'])

def cap_outliers(series, q=0.99):
    cap = series.quantile(q)
    return np.where(series > cap, cap, series)

df['Demand_log'] = (
    df.groupby(['Category','Region','Store ID','Product ID'])['Demand_log']
      .transform(cap_outliers)
)

In [ ]:
for lag in [1, 7, 14]:
    df[f'lag_{lag}'] = (
        df.groupby(['Category','Region','Store ID','Product ID'])['Demand_log']
          .shift(lag)
    )

In [ ]:
df['day'] = df['Date'].dt.day
df['month'] = df['Date'].dt.month
df['dayofweek'] = df['Date'].dt.dayofweek

In [ ]:
def train_test_split_ts(df, test_ratio=0.2):
    split = int(len(df) * (1 - test_ratio))
    return df.iloc[:split], df.iloc[split:]

In [ ]:
#holt winter
# No longer need to initialize hw_results as a separate list

configs = [('add','add'), ('add','mul'), ('mul','add'), ('mul','mul')]

for (cat, reg, store, prod), grp in df.groupby(
    ['Category','Region','Store ID','Product ID']
):

    ts = grp.set_index('Date')['Demand_log'].dropna()

    if len(ts) < 40:
        continue

    train, test = ts_split(ts)

    best_mape = np.inf

    for trend, season in configs:
        try:
            model = ExponentialSmoothing(
                train,
                trend=trend,
                seasonal=season,
                seasonal_periods=7
            ).fit()

            pred = np.expm1(model.forecast(len(test)))
            actual = np.expm1(test)

            mape = mean_absolute_percentage_error(actual, pred)
            r2 = r2_score(actual, pred)

            if mape < best_mape:
                best_mape = mape
                best_model = (trend, season, mape, r2)

        except:
            continue

    if best_mape < np.inf:
        all_results.append({
            'Model': 'Holt-Winters (Fine-Tuned)',
            'Category': cat,
            'Region': reg,
            'Store ID': store,
            'Product ID': prod,
            'MAPE': best_model[2],
            'R2': best_model[3]
        })